# Train my own GPT-2 in ZH_TW

這份筆記，是作為自己實作 GPT-2，並利用繁體中文語料訓練的記錄。


## Generative Pre-trained Transformer (GPT)

Generative Pretrained Transformer (GPT) 是 [OpenAI](https://openai.com/blog/better-language-models/) 開發的一個模型，是目前自然語言處理 (Natural Language Processing, NLP) 領域的 SOTA (state-of-the-art) 模型。2020 年所發表的 [GPT-3](https://en.wikipedia.org/wiki/GPT-3)，光是模型本身就包含了 175B 個參數，除了模型自動產生的文章已經與真人所寫的無異之外，在各種 NLP 的作業上也都取得了不錯的成績，甚至還有研究人員用它來解微分方程。

由於 GPT-3 的模型太過龐大，也衍生出很多其他的討論，像是[這篇](https://buzzorange.com/techorange/2020/08/25/gpt-3-ai-model/)文章引用的討論，估計雲端運算的成本約1200萬美金，也預估未來這類模型只有有錢的大企業才能玩得起。本文作為實驗記錄，我們也無須砸大錢去實作 GPT-3，因此我們從有[開源的 GPT-2](https://github.com/openai/gpt-2) 開始實作，並且測試在繁體中文文本上的可行性。

事實上 GPT-2 發佈以來，已經有一些簡體中文的 porting，像是：
- [gpt2-chinese](https://github.com/Morizeyao/GPT2-Chinese)
- [gpp2-ml](https://github.com/imcaspar/gpt2-ml)

經過初步研究，OpenAI 的 GPT-2 是建立於 TensorFlow 1.4，因此上述的 repo 也是建立在相同的基礎上。而撰寫本文的現在，最新的 TensorFlow 是 2.4 版， TF 在 2.0 之後有重大的改版，所以我們可能需要從頭檢視並修改原始的模型。


## Transformer Model

Vaswani 等人在 2017 年提出了注意力機制和 transformer model (Vaswani et al., 2017)，改善了用 RNN 處理語言模型的幾個問題，而這個 transformer model 也是 GPT 的基礎。關於 Transformer model 的介紹，可以參考以下兩個資訊：

- [The illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) by Jay Alammar
- [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) by Harvard NLP

簡單的說，transformer model 是加入了 attention mechanism 的 encoder-decoder 架構，在這個測試裡，我們先直接使用 [Hugging Face](https://huggingface.co/) 提供的 GPT-2 implementation，以及自己準備的語料來訓練。



### 參考資料
- Arshabhi Kayal, 2020. [Train GPT-2 in your own language](https://towardsdatascience.com/train-gpt-2-in-your-own-language-fc6ad4d60171):A step-by-step guide to train your own GPT-2 model for text generation in your choice of language from scratch

- Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, L., & Polosukhin, I. (2017). **Attention Is All You Need**. ArXiv:1706.03762 [Cs]. [http://arxiv.org/abs/1706.03762](http://arxiv.org/abs/1706.03762)

- Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). **Language Models are Unsupervised Multitask Learners**. /paper/Language-Models-are-Unsupervised-Multitask-Learners-Radford-Wu/9405cc0d6169988371b2755e573cc28650d14dfe

- Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., Agarwal, S., Herbert-Voss, A., Krueger, G., Henighan, T., Child, R., Ramesh, A., Ziegler, D. M., Wu, J., Winter, C., … Amodei, D. (2020). **Language Models are Few-Shot Learners**. ArXiv:2005.14165 [Cs]. http://arxiv.org/abs/2005.14165



## 1. 準備資料

訓練語料的品質對於訓練結果影響很大，我們先用維基百科的中文語料來做訓練。

維基百科的最新的中文語料 dump 可以透過以下連結取得：`https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2`，或是用以下的script `wikipedia_download.py`：

> python wikipedia_download.py --lang zh

以 2021-02-18 為例，中文語料的壓縮檔下載大小約為 2.0G，此 script 會下載壓縮的語料，並透過 `gensim.corpora.WikiCorpus` 將下載的語料依據文章編號切割為分開的純文字檔案（約20萬篇），存放在 `zh_corpus` 資料夾中。

In [ ]:
import tensorflow as tf
from gensim.corpora import WikiCorpus
import os
import argparse

# 定義語言為中文
lang = 'zh'


def store(corpus, lang):
    base_path = os.getcwd()
    store_path = os.path.join(base_path, '{}_corpus'.format(lang))
    if not os.path.exists(store_path):
        os.mkdir(store_path)
    file_idx=1
    for text in corpus.get_texts():
        current_file_path = os.path.join(store_path, 'article_{}.txt'.format(file_idx))
        with open(current_file_path, 'w' , encoding='utf-8') as file:
            file.write(bytes(' '.join(text), 'utf-8').decode('utf-8'))
        #endwith
        file_idx += 1
    #endfor

def tokenizer_func(text: str, token_min_len: int, token_max_len: int, lower: bool) -> list:
    return [token for token in text.split() if token_min_len <= len(token) <= token_max_len]


def run(lang):
    origin='https://dumps.wikimedia.org/{}wiki/latest/{}wiki-latest-pages-articles.xml.bz2'.format(lang,lang)
    fname='{}wiki-latest-pages-articles.xml.bz2'.format(lang)
    file_path = tf.keras.utils.get_file(origin=origin, fname=fname, untar=False, extract=False)
    corpus = WikiCorpus(file_path, lemmatize=False, lower=False, tokenizer_func=tokenizer_func)
    store(corpus, lang)

    
if __name__ == '__main__':
    ARGS_PARSER = argparse.ArgumentParser()
    ARGS_PARSER.add_argument(
        '--lang',
        default='en',
        type=str,
        help='language code to download from wikipedia corpus'
    )
    ARGS = ARGS_PARSER.parse_args()
    run(**vars(ARGS))